In [1]:
import numpy as np
import keras
import cv2
import glob
import functools

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils

from sklearn.model_selection import train_test_split

from PIL import Image as PImage

from os import listdir



Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
 colab					    findapp.gsheet
'Copy of DK - Big ideas simply explained'  'Untitled presentation.gslides'
 dataset


In [0]:
data = np.load("/content/drive/My Drive/colab/data224.npy")
labels = np.load("/content/drive/My Drive/colab/label.npy")


In [0]:
labels = keras.utils.to_categorical(labels,10)

In [0]:
(trainX, testX, trainY, testY) = train_test_split(np.array(data),np.array(labels), test_size=0.25)

In [6]:
vgg = keras.applications.vgg16.VGG16(include_top=True, weights="imagenet")

for layer in vgg.layers:
  layer.trainable = False
  
softmax = Dense(10,activation='softmax')(vgg.layers[-2].output)

model = Model(vgg.input,softmax)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
553467904/553467096 [==============================] - 21s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
__

In [7]:

top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)
top3_acc.__name__ = 'top3_acc'

top1_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=1)
top1_acc.__name__ = 'top1_acc'

model.compile(loss="categorical_crossentropy", optimizer='Adam',
	metrics=["accuracy",top3_acc, top1_acc])

H = model.fit(trainX, trainY, validation_data=(testX, testY),
epochs=5, batch_size=32)

Instructions for updating:
Use tf.cast instead.
Train on 9101 samples, validate on 3034 samples
Epoch 1/5
9101/9101 [==============================] - 125s 14ms/step - loss: 0.5327 - acc: 0.8678 - top3_acc: 0.9628 - top1_acc: 0.8678 - val_loss: 0.4433 - val_acc: 0.8889 - val_top3_acc: 0.9753 - val_top1_acc: 0.8889
Epoch 2/5
9101/9101 [==============================] - 115s 13ms/step - loss: 0.2573 - acc: 0.9288 - top3_acc: 0.9885 - top1_acc: 0.9288 - val_loss: 0.4372 - val_acc: 0.8909 - val_top3_acc: 0.9799 - val_top1_acc: 0.8909
Epoch 3/5
9101/9101 [==============================] - 115s 13ms/step - loss: 0.1436 - acc: 0.9548 - top3_acc: 0.9957 - top1_acc: 0.9548 - val_loss: 0.6075 - val_acc: 0.8754 - val_top3_acc: 0.9723 - val_top1_acc: 0.8754
Epoch 4/5
9101/9101 [==============================] - 115s 13ms/step - loss: 0.0864 - acc: 0.9701 - top3_acc: 0.9987 - top1_acc: 0.9701 - val_loss: 0.4536 - val_acc: 0.8968 - val_top3_acc: 0.9779 - val_top1_acc: 0.8968
Epoch 5/5
9101/9101 [===